In [161]:
import numpy as np
import copy 

In [162]:
T = [0, 15] # terminal states

#Matrix Population

#Actions
# 0 = up
# 1 = left
# 2 = down
# 3 = right

P = np.zeros((16, 4, 16, 2)) #p(s', r|s, a) distribution
rc_size = 4
rewards = [0, -1]
for state in range(rc_size * rc_size):
    for action in range(4):
        if state != T[0] or state != T[1]:
            u_x = state//rc_size
            u_y = state%rc_size
            #print(u_x * rc_size + u_y)
            if action == 0:
                s_ = (u_x - 1) * rc_size + u_y
                if s_ == T[0] or s_ == T[1]:
                    P[state][action][s_][0] += 1
                elif u_x - 1 >= 0:
                    P[state][action][s_][1] += 1
                else:
                    P[state][action][state][1] += 1
            elif action == 1:
                s_ = (u_x * rc_size) + (u_y - 1)
                if s_ == T[0] or s_ == T[1]:
                    P[state][action][s_][0] += 1
                elif u_y - 1 >= 0:
                    P[state][action][s_][1] += 1
                else:
                    P[state][action][state][1] += 1
            elif action == 2:
                s_ = (u_x + 1) * rc_size + u_y
                if s_ == T[0] or s_ == T[1]:
                    P[state][action][s_][0] += 1
                elif u_x + 1 < rc_size:
                    P[state][action][s_][1] += 1
                else:
                    P[state][action][state][1] += 1
            elif action == 3:
                s_ = (u_x * rc_size) + (u_y + 1)
                if s_ == T[0] or s_ == T[1]:
                    P[state][action][s_][0] += 1
                elif u_y + 1 < rc_size:
                    P[state][action][s_][1] += 1
                else:
                    P[state][action][state][1] += 1

                    

In [163]:
#Code for Optimal Values and Policy Generation using Policy Iteration Method
def next_lookup(Values, state, gamma):
    V_actions = np.zeros(4)
    for action in range(len(V_actions)):
        for otherstate in range(rc_size*rc_size):
            for reward in range(len(rewards)):
                V_actions[action] += P[state][action][otherstate][reward]*(rewards[reward] + gamma*Values[otherstate])
    
    return V_actions
    

def policy_evaluation(Values, Policy, gamma = 1.0, theta = 0.1):
    while True:
        #count += 1
#        Updated = np.zeros(rc_size * rc_size)
        delta = 0
        for state in range(1, rc_size*rc_size-1, 1):
            b_value = 0
            for action in range(4):
                for otherstate in range(rc_size*rc_size):
                    for reward in range(len(rewards)):
                        b_value += Policy[state][action] * P[state][action][otherstate][reward]*(rewards[reward] + gamma * Values[otherstate])
                
            delta = max(delta, np.abs(Values[state] - b_value))
            Values[state]= b_value
#        for state in range(rc_size*rc_size):
#            Values[state] = Updated[state]
        if delta <= theta:
            break
    #print(count)
    return Values


def Policy_Iteration(gamma = 1.0, theta = 0.1):
    policy_state = True
    Values = np.zeros(rc_size * rc_size)
    Policy = np.zeros((rc_size * rc_size, 4))
    Policy[1:len(Policy)-1] += 0.25
    while True:
        policy_state = True
        Values = policy_evaluation(Values, Policy, gamma, theta)
        for state in range(1, rc_size * rc_size - 1, 1):
            old_action = copy.deepcopy(Policy[state])
            V_actions = next_lookup(Values, state, gamma)
            m = np.max(V_actions)
            count = np.count_nonzero(V_actions == m)
            updated_p = 1/count
            for i in range(len(V_actions)):
                if V_actions[i] == m:
                    Policy[state][i] = updated_p
                else:
                    Policy[state][i] = 0
            if np.array_equal(old_action,  Policy[state]) == False:
                policy_state = False
        print(np.around(Values.reshape(4,4), decimals = 2))
        print(getReadablePolicies(Policy))
        print("=========")
        
        if policy_state == True:
            break
    return Values, Policy

def getReadablePolicies(Policies):
    Policy = []
    for i in range(len(Policies)):
        s = ""
        for j in range(4):
            if j == 0 and Policies[i][j] > 0:
                s = s + "U"
            if j == 1 and Policies[i][j] > 0:
                s = s + "L"
            if j == 2 and Policies[i][j] > 0:
                s = s + "D"
            if j == 3 and Policies[i][j] > 0:
                s = s + "R"
        Policy.append(s)
    return Policy
          

In [158]:
def policy_evaluation1(Policy, gamma = 1.0, theta = 0.1):
    Values = np.zeros(rc_size * rc_size) 
    Updated = np.zeros(rc_size * rc_size)
    for i in range(100):
        for state in range(rc_size*rc_size):
            b_value = 0
            for action in range(4):
                for otherstate in range(1, rc_size*rc_size-1, 1):
                    for reward in range(len(rewards)):
                        b_value += Policy[state][action] * P[state][action][otherstate][reward]*(rewards[reward] + gamma * Values[otherstate])
                
            #delta = max(delta, np.abs(Values[state] - b_value))
            Values[state]= b_value
        #Values = copy.deepcopy(Updated)

    return Values

In [159]:
Values, Policy = Policy_Iteration(1, 0.0001)

[[  0. -13. -19. -21.]
 [-13. -17. -19. -19.]
 [-19. -19. -17. -13.]
 [-21. -19. -13.   0.]]
['', 'L', 'L', 'L', 'U', 'UL', 'L', 'D', 'U', 'U', 'D', 'D', 'U', 'R', 'R', '']
[[ 0.  0. -1. -2.]
 [ 0. -1. -2. -1.]
 [-1. -2. -1.  0.]
 [-2. -1.  0.  0.]]
['', 'L', 'L', 'LD', 'U', 'UL', 'ULDR', 'D', 'U', 'ULDR', 'DR', 'D', 'UR', 'R', 'R', '']
[[ 0.  0. -1. -2.]
 [ 0. -1. -2. -1.]
 [-1. -2. -1.  0.]
 [-2. -1.  0.  0.]]
['', 'L', 'L', 'LD', 'U', 'UL', 'ULDR', 'D', 'U', 'ULDR', 'DR', 'D', 'UR', 'R', 'R', '']


In [160]:
print(np.around(Values.reshape(4,4), decimals = 3))
print()
print(getReadablePolicies(Policy))
print(policy_evaluation1(Policy).reshape(4,4))

[[ 0.  0. -1. -2.]
 [ 0. -1. -2. -1.]
 [-1. -2. -1.  0.]
 [-2. -1.  0.  0.]]

['', 'L', 'L', 'LD', 'U', 'UL', 'ULDR', 'D', 'U', 'ULDR', 'DR', 'D', 'UR', 'R', 'R', '']
[[ 0.  0. -1. -2.]
 [ 0. -1. -2. -1.]
 [-1. -2. -1.  0.]
 [-2. -1.  0.  0.]]
